Install Necessary Nuget packages

In [ ]:
#region install Nuget
//Essential libs for running .NET 6 Generic Host
#r "nuget: Microsoft.Extensions.Hosting"
#r "nuget: Microsoft.Extensions.DependencyInjection"

//Orleans essential dependencies
#r "nuget: Microsoft.Orleans.Core"
#r "nuget: Microsoft.Orleans.OrleansRuntime"
//Orleans silo server runtime essentials
#r "nuget: Microsoft.Orleans.Server"
#endregion


Installed Packages Microsoft.Extensions.DependencyInjection, 6.0.0 Microsoft.Extensions.Hosting, 6.0.1 Microsoft.Orleans.Core, 3.6.5 Microsoft.Orleans.OrleansRuntime, 3.6.5 Microsoft.Orleans.Server, 3.6.5

Load some namespaces for .NET Generic Host & Orleans extension method(s).

In [ ]:
using Microsoft.Extensions.Hosting;
using Orleans;
using Orleans.Runtime;
using Orleans.Hosting;

Manually compile Orleans Grain class library project:

In [ ]:
#!pwsh
dotnet build ../src/Grains/RpcDemo.Grains.Greeting/RpcDemo.Grains.Greeting.csproj --nologo --verbosity quiet


Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:24.72


Load builded assembly files:

In [ ]:
#r "../src/Shared/RpcDemo.Interfaces.Hello/bin/Debug/netstandard2.0/RpcDemo.Interfaces.Hello.dll"
#r "../src/Grains/RpcDemo.Grains.Greeting/bin/Debug/net6.0/RpcDemo.Grains.Greeting.dll"

Craft Orleans Server side builder code:

In [ ]:
using RpcDemo.Interfaces.Hello;
using RpcDemo.Grains.Greeting;
var hostBuilder = Host.CreateDefaultBuilder()
    .UseOrleans(builder =>
    {
        builder.UseLocalhostClustering();
        builder.ConfigureApplicationParts(parts => {
              parts.AddApplicationPart(typeof(IHelloGrain).Assembly).WithReferences();
              parts.AddApplicationPart(typeof(HelloGrain).Assembly).WithReferences();
            });
    });

Create .NET Generic Host by calling `Build()`, and then start it:

In [ ]:
var host = hostBuilder.Build();
await host.StartAsync();

info: Orleans.Runtime.Silo[100404]
      Silo starting with GC settings: ServerGC=True GCLatencyMode=Interactive
info: Orleans.Runtime.Silo[100403]
      -------------- Initializing silo on host vm-ubuntu2204 MachineName vm-ubuntu2204 at 127.0.0.1:11111, gen 401534855 --------------
info: Orleans.Runtime.Silo[100415]
      Starting silo Silo_b67ab
warn: Orleans.Runtime.NoOpHostEnvironmentStatistics[100708]
      No implementation of IHostEnvironmentStatistics was found. Load shedding will not work yet
info: Orleans.Runtime.ConsistentRing.VirtualBucketsRingProvider[0]
      Starting VirtualBucketsRingProvider on silo S127.0.0.1:11111:401534855/xB30C4E58.
info: Orleans.Runtime.ConsistentRing.VirtualBucketsRingProvider[103003]
      Added Server S127.0.0.1:11111:401534855/xB30C4E58. Current view: [S127.0.0.1:11111:401534855 -> <MultiRange: Size=x100000000, %Ring=100.000%>]
info: Orleans.Runtime.ConsistentRing.VirtualBucketsRingProvider[103005]
      -NotifyLocalRangeSubscribers about old 

Craft client builder code:

In [ ]:
var clientBuilder = new ClientBuilder().UseLocalhostClustering();
clientBuilder.ConfigureApplicationParts(parts=>parts.AddApplicationPart(typeof(IHelloGrain).Assembly));

In [ ]:
var client = clientBuilder.Build();

Connect to Silo:

In [ ]:
await client.Connect();

Call `GetGrain<IHelloGrain>()` to get RPC proxy:

In [ ]:
var helloGrainRpc = client.GetGrain<IHelloGrain>(0);

Call RPC method:

In [ ]:
var greeting = await helloGrainRpc.SayHello("Orleans");
display(greeting);

Hello Orleans!

Remember to call `Close()` when client-side don't need to use anymore.

In [ ]:
await client.Close();

Shutdown server by calling `StopAsync()`:

In [ ]:
await host.StopAsync().ConfigureAwait(false);